In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import os

In [2]:
input_file = 'Weighted_paths_0.65' # this is the 'output_file_1.evaluated.filtered_0.67' file
ranking_file = './ranks/ranking_0.65'
N = 100 # top ranked genes in the network
output_file = 'subnetwork_0.65'+str(N)+'.js'
output_file_components_directory = './components/subnetwork_0.65_'+str(N)+'/'

In [3]:
df = pd.read_csv(input_file,sep='[',skiprows=0,header=None)

In [4]:
df = df.rename(columns={0: 'path_weight', 1: 'genes_path'})

In [5]:
df

,path_weight,genes_path
0,4.3260559891078766e-9-,"'BIRC6','EP300','MED1']."
1,4.326456975658613e-9-,"'ETS1','EP300','SMAD2']."
2,4.342250895728172e-9-,"'LTBR','EP300','RB1']."
3,4.344618133181321e-9-,"'TF','EP300','RB1']."
4,4.344732622970707e-9-,"'MAGEA6','EP300','RB1']."
...,...,...
6158594,0.00035902277744005495-,"'CLCNKA','BSND','CLCNKB']."
6158595,0.0003598853280061405-,"'ATP4A','ATP4B','ATP12A']."
6158596,0.00041027515798344706-,"'EZH2','CHD2','CHD6']."
6158597,0.00041468980120503004-,"'EP300','ARID5B','PHF2']."


In [6]:
df[['gene1','gene2','gene3']] = df['genes_path'].str.split(',',expand=True)
df[['gene3','brol']] = df['gene3'].str.split(']',expand=True)
df = df.drop(['brol','genes_path'], axis=1)
df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)
df = df.drop(['brol'], axis=1)
df

/tmp/ipykernel_7449/271992433.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,4.3260559891078766e-9,'BIRC6','EP300','MED1'
1,4.326456975658613e-9,'ETS1','EP300','SMAD2'
2,4.342250895728172e-9,'LTBR','EP300','RB1'
3,4.344618133181321e-9,'TF','EP300','RB1'
4,4.344732622970707e-9,'MAGEA6','EP300','RB1'
...,...,...,...,...
6158594,0.00035902277744005495,'CLCNKA','BSND','CLCNKB'
6158595,0.0003598853280061405,'ATP4A','ATP4B','ATP12A'
6158596,0.00041027515798344706,'EZH2','CHD2','CHD6'
6158597,0.00041468980120503004,'EP300','ARID5B','PHF2'


In [7]:
df_ranking = pd.read_csv(ranking_file,sep='[',skiprows=0,header=None)

In [8]:
df_ranking_top = df_ranking[0:N]
df_ranking_top
top_genes = df_ranking_top[0].values.tolist()
top_genes2 = []
for i in top_genes:
    a = '\''+i+'\''
    top_genes2.append(a)

In [9]:
df_final = df[(df['gene1'].isin(top_genes2) & df['gene2'].isin(top_genes2)) | (df['gene1'].isin(top_genes2) & df['gene3'].isin(top_genes2)) | (df['gene2'].isin(top_genes2) & df['gene3'].isin(top_genes2))]
df_final

,path_weight,gene1,gene2,gene3
0,4.3260559891078766e-9,'BIRC6','EP300','MED1'
18,4.410498012649389e-9,'SMARCD2','EP300','CAMK2A'
19,4.415897701127988e-9,'CABIN1','EP300','MED1'
20,4.416217037227601e-9,'AXIN2','EP300','MED1'
21,4.416262345564375e-9,'REST','EP300','MED1'
...,...,...,...,...
6156169,1.210583117489051e-5,'ITGB2','C3','CFHR5'
6156171,1.2113322176734077e-5,'ITGB2','C3','CFHR2'
6156282,1.249831494156447e-5,'ITGAM','C3','CFHR1'
6156284,1.2505853483022103e-5,'ITGAM','C3','CFHR5'


In [10]:
edges = {}
Normalization = sum(df['path_weight'].astype(float)) # Explained in the paper of OMEN
for i in range(N):
    print(i)
    for j in range(N-1-i):
        node1 = top_genes[i]
        node2 = top_genes[i+1+j]
        nodes = []
        nodes.append(top_genes2[i])
        nodes.append(top_genes2[i+1+j])
        df_edge = df_final[ (df_final['gene1'].isin(nodes) & df_final['gene2'].isin(nodes)) | (df_final['gene1'].isin(nodes) & df_final['gene3'].isin(nodes)) | (df_final['gene2'].isin(nodes) & df_final['gene3'].isin(nodes))]
        s = sum(df_edge['path_weight'].astype(float))/Normalization
        if s>0:
            edges[(node1,node2)] = s*100000 # to have bigger values

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
print(len(edges))
edges

2832


{('EP300', 'TP53'): 7.417184086282755,
 ('EP300', 'STAT1'): 12.103049988645209,
 ('EP300', 'PRKACA'): 6.905339225267018,
 ('EP300', 'CTNNB1'): 5.757756473088021,
 ('EP300', 'CTCF'): 8.304336362531648,
 ('EP300', 'STAT3'): 6.008511150256149,
 ('EP300', 'GPLD1'): 0.10246530247070469,
 ('EP300', 'FOXA1'): 7.216029167208216,
 ('EP300', 'KCNAB1'): 0.06173655505009993,
 ('EP300', 'AR'): 7.380702244359456,
 ('EP300', 'AKT1'): 4.741948148105418,
 ('EP300', 'PRKCB'): 4.6537667126265925,
 ('EP300', 'KRAS'): 4.682985396037567,
 ('EP300', 'TCF12'): 7.166343322589058,
 ('EP300', 'RELA'): 4.6942117155074605,
 ('EP300', 'CDC42'): 5.2560017709094655,
 ('EP300', 'MYC'): 5.017432494635979,
 ('EP300', 'ESR1'): 6.187138317434229,
 ('EP300', 'RHOC'): 0.3450782563633306,
 ('EP300', 'SP3'): 0.41910952335266133,
 ('EP300', 'C3'): 0.4159552912154362,
 ('EP300', 'KCNAB2'): 0.06248943687105663,
 ('EP300', 'SMAD4'): 4.84242625204455,
 ('EP300', 'PRKACB'): 4.589895302700698,
 ('EP300', 'PRKACG'): 4.60918955532491,

In [12]:
with open(output_file,'w') as f:
    f.write('graph = {')
    f.write('\n')
    f.write('    nodes: [')
    f.write('\n')
    for i in range(N):
        f.write('        {') 
        f.write('\n')
        f.write('            id: ' + top_genes2[i] + ',') 
        f.write('\n')
        f.write('            rank: ' + str(1+i) + ',') 
        f.write('\n')
        f.write('        },') 
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('    links: [')
    f.write('\n')
    for edge in edges:
        f.write('        {source: "'+ edge[0] +'", target: "'+edge[1]+'", type: "pp", direction: "directed", max_cost: '+ str(edges[edge]) +', evidence: ""},')
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('}')
f.close()

In [13]:
def componentsFilteredEdges(edges,Filter):
    array1 = []
    for edge in edges:
        if edges[edge]>Filter:
            array1.append(edge)
    G = nx.Graph()
    G.add_edges_from(array1)
    r = list(nx.connected_components(G))
    return r

In [14]:
def writeComponents(edges,Filters):
    for f in Filters:
        components = componentsFilteredEdges(edges,f)
        file = 'components_' + str(f)
        with open(output_file_components_directory+file,'w') as f:
            i = 1
            for c in components:
                f.write('Component'+str(i)+': ')
                f.write(str(c))
                f.write('\n')
                i = i + 1

In [15]:
path = os.path.join(output_file_components_directory)
os.mkdir(path)
writeComponents(edges,[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10,10.5,11,11.5,12,12.5,13])